In [1]:
import os
import pandas as pd
import numpy as np

# 配置
input_base_dir = r'ZTF_SN_Data'
output_base_dir = r'ZTF_SN_FLUX'
os.makedirs(output_base_dir, exist_ok=True)

def mag_to_snana_flux(mag, mag_err=None):
    """将mag转换为SNANA标准定义的flux和flux_err"""
    alpha = 4 * np.log(10)  # 9.21034
    flux = 10**(-0.4 * mag + 11)

    if mag_err is not None:
        flux_err = 10**10 * alpha * mag_err * np.exp(-alpha * mag / 10)
        return flux, flux_err
    else:
        return flux

# 遍历所有源
for source_id in os.listdir(input_base_dir):
    source_path = os.path.join(input_base_dir, source_id)
    if os.path.isdir(source_path):
        # 创建对应的新目录
        output_source_path = os.path.join(output_base_dir, source_id)
        os.makedirs(output_source_path, exist_ok=True)

        for file_name in os.listdir(source_path):
            input_file = os.path.join(source_path, file_name)
            output_file = os.path.join(output_source_path, file_name)

            # 只处理csv文件（g、r、i波段的光变数据）
            if file_name.endswith('.csv'):
                try:
                    df = pd.read_csv(input_file)

                    if df.shape[1] >= 3:
                        # 假设第1列是时间，第2列是mag，第3列是mag_err
                        time_col = df.iloc[:, 0]
                        mag_col = df.iloc[:, 1]
                        mag_err_col = df.iloc[:, 2]

                        flux_col, flux_err_col = mag_to_snana_flux(mag_col, mag_err_col)

                        # 保持其他列不变，只替换第2列、第3列
                        df.iloc[:, 1] = flux_col
                        df.iloc[:, 2] = flux_err_col

                        df.to_csv(output_file, index=False)
                    else:
                        # 少于3列的直接原样拷贝
                        df.to_csv(output_file, index=False)

                except Exception as e:
                    print(f"Error processing {input_file}: {e}")

            else:
                # 其他文件（比如label.txt等），直接拷贝
                try:
                    with open(input_file, 'rb') as f_in:
                        with open(output_file, 'wb') as f_out:
                            f_out.write(f_in.read())
                except Exception as e:
                    print(f"Error copying {input_file}: {e}")

print("✅ 完成！所有数据已转换为SNANA flux格式并保存到 ZTF_SN_FLUX/")


✅ 完成！所有数据已转换为SNANA flux格式并保存到 ZTF_SN_FLUX/
